In [ ]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.checkpoint.sqlite import SqliteSaver

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable": {
        "thread_id": "0_x",
    },
}

In [ ]:
class State(MessagesState):
    pass


graph_builder = StateGraph(State)

In [ ]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response],
    }

In [ ]:
graph_builder.add_node("chatbot", chatbot)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [ ]:
result = graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I live in Europe now. And the city I live in is Valencia.",
            },
        ]
    },
    config=config,
)

In [ ]:
for message in result["messages"]:
    message.pretty_print()

In [ ]:
state_history = graph.get_state_history(config)

In [ ]:
for state_snapshot in list(state_history):
    print(state_snapshot.next)
    print(state_snapshot.values["messages"])
    print("=========\n")

In [ ]:
state_history = graph.get_state_history(config)

to_fork = list(state_history)[-5]

to_fork.config

In [ ]:
from langchain_core.messages import HumanMessage

graph.update_state(
    to_fork.config,
    {
        "messages": [
            HumanMessage(
                content="I live in Europe now. And the city I live in is Zagreb.",
                id="25169a3d-cc86-4a5f-9abd-03d575089a9f",
            )
        ]
    },
)

In [ ]:
forked_state = graph.get_state_history(
    {
        "configurable": {
            "thread_id": "11",
            "checkpoint_ns": "",
            "checkpoint_id": "1f08d808-b408-6ca2-8004-f964cbac5a14",
        }
    }
)

list(forked_state)

In [ ]:
result = graph.invoke(
    None,
    {
        "configurable": {
            "thread_id": "0_x",
            "checkpoint_ns": "",
            "checkpoint_id": "1f08d808-b408-6ca2-8004-f964cbac5a14",
        }
    },
)

for message in result["messages"]:
    message.pretty_print()